In [1]:
import keras
import numpy as np

import os
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

Using TensorFlow backend.


In [2]:
training_data = 'imagens/treinamento'
validation_data = 'imagens/validacao'

In [3]:
import pylab
import pydicom as dicom
import glob
import png
import os

#Transforma as imagens DICOM em PNG

diretorio_list = ['imagens/treinamento/positivo/', 'imagens/validacao/positivo/',
                  'imagens/treinamento/negativo/', 'imagens/validacao/negativo/']
for diretorio in diretorio_list:
    for img in glob.glob(diretorio + '*.dcm'):
          mri_file = open(img, 'rb')
          imagem = dicom.read_file(mri_file)
          mri_file.close()

          shape = imagem.pixel_array.shape

          image_2d = []
          max_val = 0
          for row in imagem.pixel_array:
              pixels = []
              for col in row:
                  pixels.append(col)
                  if col > max_val: max_val = col
              image_2d.append(pixels)

          image_2d_scaled = []
          for row in image_2d:
              row_scaled = []
              for col in row:
                  col_scaled = int((float(col) / float(max_val)) * 255.0)
                  row_scaled.append(col_scaled)
              image_2d_scaled.append(row_scaled)

          png_nome = os.path.basename(img)[:-3] + 'png'
          print(png_nome)
          png_file = open(diretorio+png_nome, 'wb')
          w = png.Writer(shape[1], shape[0], greyscale=True)
          w.write(png_file, image_2d_scaled)
          png_file.close()


000008.png
000007.png
000006.png
000008.png
000007.png
000006.png
000008.png
000007.png
000006.png


In [4]:
img_width, img_height = 512, 512

# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        training_data,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')


Found 6 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [6]:
#Construindo o modelo 
# Initialising the CNN
model = Sequential()

# Convolution
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Terceira camada
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())


# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

/home/gabriela/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(512, 512,...)`
  
/home/gabriela/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  del sys.path[0]
/home/gabriela/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [17]:
#Treinamento

nb_epochnb_epoc  = 30
nb_train_samples = 100
nb_validation_samples = 80

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)
model.save_weights('modelos/basic_cnn_1.h5')

/home/gabriela/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
/home/gabriela/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=6, epochs=10, validation_steps=80)`
  if sys.path[0] == '':


Epoch 1/10
6/6 [==============================] - 113s 19s/step - loss: 4.5425 - acc: 0.6389 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/10
6/6 [==============================] - 39s 7s/step - loss: 5.3141 - acc: 0.6667 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/10
6/6 [==============================] - 43s 7s/step - loss: 5.3141 - acc: 0.6667 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
6/6 [==============================] - 41s 7s/step - loss: 5.3141 - acc: 0.6667 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/10
6/6 [==============================] - 46s 8s/step - loss: 2.9457 - acc: 0.5833 - val_loss: 0.6310 - val_acc: 1.0000
Epoch 6/10
6/6 [==============================] - 39s 7s/step - loss: 0.6561 - acc: 0.6389 - val_loss: 0.5957 - val_acc: 0.7500
Epoch 7/10
6/6 [==============================] - 40s 7s/step - loss: 0.4221 - acc: 0.8611 - val_loss: 0.4355 - val_acc: 0.7500
Epoch 8/10
6/6 [==============================] - 40s 7s/step - loss: 0.0918 - acc: 1.0000 - val_loss:

OSError: Unable to create file (unable to open file: name = 'models/basic_cnn_1.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [28]:
#Teste
img_teste = keras.preprocessing.image.load_img('imagens/teste/000001.png')
prediction = model.predict(img_teste)
print(prediction)

AttributeError: 'Image' object has no attribute 'ndim'